In [23]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
%%writefile app.py  
# Creamos el archivo de la APP en el intérprete principal (Python)

############################# IMPLEMENTACIÓN DE DASHBOARD ################################

# Importamos librerías
import plotly.graph_objects as go
import streamlit as st
import seaborn as sns
import plotly.express as px
import pandas as pd
import numpy as np
from funpymodeling.exploratory import freq_tbl 
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures



#################################################################

# Definimos la instancia de streamlit
@st.cache_resource

#################################################################

# Creamos la función de carga de datos
def load_data():
    # Lectura del archivo CSV sin índice
    df1 = pd.read_csv("data_limpio - Chicago 1.csv")
    
    # Etapa de procesamiento de Datos
    
    # Limpiar la columna 'price' y convertirla a numérica
    df1['price'] = pd.to_numeric(df1['price'], errors='coerce')  # Convierte a numérico, convierte errores a NaN

    # Filtrar filas donde 'price' no es NaN
    df1 = df1[df1['price'].notna()]
    
    # ANÁLISIS UNIVARIADO DE FRECUENCIAS
    # Obtengo un análisis univariado de una variable categórica en específico
    table = freq_tbl(df1['property_type'])
    
    # Obtengo un filtro de los valores más relevantes de la variable categórica seleccionada
    Filtro = table[table['frequency'] > 1]
    
    # Ajusto el índice de mi dataframe
    Filtro_index1 = Filtro.set_index('property_type')
    
    # Selecciono las columnas tipo numéricas del dataframe df1
    numeric_df1 = df1.select_dtypes(['float', 'int'])  # Devuelve Columnas
    numeric_cols1 = numeric_df1.columns                # Devuelve lista de Columnas 

    return Filtro_index1, df1, numeric_df1, numeric_cols1

#################################################################

# Cargo los datos obtenidos de la función "load_data"
Filtro_index1, df1, numeric_df1, numeric_cols1 = load_data()

###################### CREACIÓN DEL DASHBOARD ################################

# 1. CREACIÓN DE LA SIDEBAR
# Generamos los encabezados para la barra lateral (sidebar)
st.sidebar.title("DASHBOARD")
st.sidebar.header("Panel de selección")
st.sidebar.subheader("Opciones")

# Crear selectbox para escoger ciudad (en este caso, solo Chicago)
city_selected = st.sidebar.selectbox("Selecciona una ciudad", ["Chicago"])

# 4 FRAMES: Frame 1, Frame 2, Frame 3, Frame 4
frames = st.sidebar.selectbox(label="Selecciona un frame", options=["Frame 1", "Frame 2", "Frame 3", "Frame 4", "Frame 5"])

# FRAME 1: Análisis Univariado y Tabla
if frames == "Frame 1":
    st.title(f"Evaluación de {city_selected} - Frame 1")
    st.header("Análisis Univariado de Tipo de Propiedad")
    
    # Gráfico de Análisis Univariado (Frecuencia)
    var_selected = st.sidebar.selectbox("Selecciona una variable para el análisis", options=["accommodates", "beds", "bedrooms", "bathrooms", "property_type", "host_location"])
    
    # Gráfico de barras para el análisis univariado con un estilo más claro
    bar_data = freq_tbl(df1[var_selected])
    barplot_figure = px.bar(
        bar_data, 
        x=var_selected, 
        y='frequency', 
        labels={var_selected: "Tipo de Propiedad", 'frequency': "Frecuencia"},
        title=f'Frecuencia de {var_selected}',
        template='plotly_white',  # Estilo más claro y moderno
        color_discrete_sequence=px.colors.qualitative.Pastel  # Colores más suaves y estéticos
    )
    barplot_figure.update_layout(
        title_font=dict(size=20, family='Arial', color='#333'),
        xaxis_title="Tipo de Propiedad",  # Etiqueta del eje X más clara
        yaxis_title="Frecuencia",         # Etiqueta del eje Y más clara
        plot_bgcolor='rgba(0,0,0,0)',     # Fondo blanco
        paper_bgcolor='#F0F2F6',          # Fondo suave del dashboard
        font_color='#333'                 # Color de fuente más claro
    )
    st.plotly_chart(barplot_figure)

    # Mostrar tablas
    st.subheader("Tabla de Frecuencia")
    st.write(Filtro_index1)

    st.subheader("Tabla de Datos Originales")
    # Filtro por tipo de propiedad
    property_filter = st.sidebar.selectbox("Filtrar por tipo de propiedad", options=df1['property_type'].unique())
    if property_filter:
        filtered_data = df1[df1['property_type'] == property_filter]
        st.write(filtered_data)

# FRAME 2: Gráficos de Barras, Scatterplot y Pieplot
elif frames == "Frame 2":
    st.title(f"Evaluación de {city_selected} - Frame 2")
    st.header("Gráficos Comparativos")
    
    # Variables permitidas para el barplot y scatterplot
    valid_vars = ["accommodates", "beds", "bedrooms", "bathrooms", "property_type", "host_location"]
    
    # Seleccionar variable para el eje y del barplot y scatterplot
    y_var_selected = st.sidebar.selectbox("Selecciona la variable Y para Barplot y Scatterplot", options=valid_vars)

    # BARPLOT
    st.subheader("Barplot por Tipo de Habitación")
    if y_var_selected:
        barplot_figure = px.bar(
            df1, 
            x='room_type', 
            y=y_var_selected, 
            title=f'{y_var_selected} por Tipo de Habitación', 
            color_discrete_sequence=px.colors.sequential.Plasma  # Cambiamos la paleta a Plasma
        )
        barplot_figure.update_layout(
            xaxis_title="Tipo de Habitación",
            yaxis_title=f'{y_var_selected}',
            font=dict(size=12),
            xaxis_tickangle=-45,  # Inclinamos las etiquetas del eje X
            plot_bgcolor='rgba(255, 255, 255, 0)',  # Fondo blanco transparente
            paper_bgcolor='#D3D3D3',  # Fondo gris claro del dashboard
            font_color='#000000',  # Color de fuente negro
            title_font=dict(size=20, family='Arial', color='#4B0082'),  # Color del título
            legend_title_font_color="blue"  # Cambia el color del título de la leyenda
        )
        st.plotly_chart(barplot_figure)

    # PIEPLOT - Solo variables específicas, excluyendo "property_type"
    st.subheader("Pieplot de Acomodaciones")
    pie_variables = ['accommodates', 'beds', 'bedrooms', 'bathrooms']
    pie_var_selected = st.sidebar.selectbox("Selecciona la variable para el Pieplot", options=pie_variables)

    if pie_var_selected:
        pieplot_figure = px.pie(
            df1, 
            names=pie_var_selected, 
            title=f'Distribución por {pie_var_selected}', 
            template='plotly_white',
            color_discrete_sequence=px.colors.qualitative.Pastel
        )
        st.plotly_chart(pieplot_figure)

# FRAME 3: Heatmap y Boxplot
elif frames == "Frame 3":
    st.title(f"Evaluación de {city_selected} - Frame 3")
    st.header("Análisis Avanzado: Correlaciones")
    
    # HEATMAP con mayor claridad
    st.subheader("Mapa de Calor de Correlaciones")
    corr_matrix = numeric_df1.corr()
    heatmap_figure = px.imshow(
        corr_matrix, 
        title="Matriz de Correlación",
        color_continuous_scale='Blues', 
        template='plotly_white',
        height=800, 
        width=800
    )
    st.plotly_chart(heatmap_figure)

 # Seleccionar variable para el boxplot
    boxplot_y_var = st.sidebar.selectbox("Selecciona la variable Y para el Boxplot", options=numeric_cols1)

    # BOXPLOT
    st.subheader("Boxplot")
    if boxplot_y_var:
        boxplot_figure = px.box(df1, x='room_type', y=boxplot_y_var, title=f'Boxplot de {boxplot_y_var} por Tipo de Habitación')
        st.plotly_chart(boxplot_figure)

# FRAME 4: Tablas y Datos Estadísticos
elif frames == "Frame 4":
    st.title(f"Evaluación de {city_selected} - Frame 4")
    st.header("Datos Estadísticos y Rangos de Precio")

    # Tabla de estadísticas descriptivas
    if st.checkbox("Mostrar Tabla de Estadísticas"):
        st.subheader("Estadísticas Descriptivas")
        st.write(df1.describe())

    # DATAFRAME con filtro por rango de precio
    st.subheader("Datos filtrados por precio")
    price_filter = st.slider("Selecciona un rango de precio", int(df1['price'].min()), int(df1['price'].max()), (50, 500))
    filtered_data = df1[(df1['price'] >= price_filter[0]) & (df1['price'] <= price_filter[1])]
    st.write(filtered_data)

    # Histograma de precios
    hist_var = st.sidebar.selectbox("Selecciona la variable para el Histograma", options=numeric_cols1)

    st.subheader(f"Distribución de {hist_var}")
    if hist_var:
        hist_figure = px.histogram(
            filtered_data, 
            x=hist_var, 
            nbins=30, 
            title=f'Distribución de {hist_var}', 
            template='plotly_white',
            color_discrete_sequence=px.colors.qualitative.Pastel
        )
        st.plotly_chart(hist_figure)

# FRAME 5: Regresiones
if frames == "Frame 5":
    st.title(f"Evaluación de {city_selected} - Frame 5")
    st.header("Regresiones")
    
    regression_type = st.sidebar.selectbox("Selecciona el tipo de regresión", options=["Regresión Lineal Simple", "Regresión Lineal Múltiple", "Regresión No Lineal", "Regresión Logística"])

    # Variables numéricas seleccionables
    x_var = st.sidebar.selectbox("Selecciona la variable X", options=numeric_cols1)
    y_var = st.sidebar.selectbox("Selecciona la variable Y", options=numeric_cols1)

    # Dividir los datos para el entrenamiento y la prueba
    X = df1[[x_var]].dropna()  # Matriz de características
    y = df1[y_var].dropna()    # Variable objetivo

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # REGRESIÓN LINEAL SIMPLE
    if regression_type == "Regresión Lineal Simple":
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Predicciones y métricas
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r_squared = model.score(X_test, y_test)

        # Gráfico
        fig = px.scatter(x=X_test[x_var], y=y_test.values.flatten(), 
                         title=f'Regresión Lineal Simple: {x_var} vs {y_var}', 
                         labels={'x': x_var, 'y': y_var}, 
                         color_discrete_sequence=['blue'])

        # Línea de predicción en rojo
        fig.add_trace(go.Scatter(x=X_test[x_var], y=y_pred, mode='lines', name='Predicción', line=dict(color='red')))

        st.plotly_chart(fig)
        st.write(f"Coeficiente de regresión: {model.coef_[0]}")
        st.write(f"Intercepto: {model.intercept_}")
        st.write(f"R²: {r_squared}")
        st.write(f"MSE: {mse}")

    # REGRESIÓN LINEAL MÚLTIPLE
    elif regression_type == "Regresión Lineal Múltiple":
        x_vars_mult = st.sidebar.multiselect("Selecciona las variables X para la regresión múltiple", options=numeric_cols1)
        if len(x_vars_mult) > 1:
            X_mult = df1[x_vars_mult].dropna()

            # Dividir los datos para entrenamiento y prueba
            X_train_mult, X_test_mult, y_train_mult, y_test_mult = train_test_split(X_mult, y, test_size=0.3, random_state=42)

            # Entrenar el modelo
            model_mult = LinearRegression()
            model_mult.fit(X_train_mult, y_train_mult)

            # Predicciones
            y_pred_mult = model_mult.predict(X_test_mult)
            mse_mult = mean_squared_error(y_test_mult, y_pred_mult)
            r_squared_mult = model_mult.score(X_test_mult, y_test_mult)

            # Mostrar resultados
            st.write(f"Coeficientes: {model_mult.coef_}")
            st.write(f"Intercepto: {model_mult.intercept_}")
            st.write(f"R²: {r_squared_mult}")
            st.write(f"MSE: {mse_mult}")
        else:
            st.write("Selecciona más de una variable para la regresión múltiple.")

    # REGRESIÓN NO LINEAL (POLINOMIAL)
    elif regression_type == "Regresión No Lineal":
        degree = st.sidebar.slider("Selecciona el grado del polinomio", min_value=2, max_value=5, value=2)

        poly = PolynomialFeatures(degree=degree)
        X_poly = poly.fit_transform(X_train)

        # Modelo de regresión lineal con características polinomiales
        model_poly = LinearRegression()
        model_poly.fit(X_poly, y_train)

        # Predicciones
        X_test_poly = poly.transform(X_test)
        y_pred_poly = model_poly.predict(X_test_poly)
        mse_poly = mean_squared_error(y_test, y_pred_poly)
        r_squared_poly = model_poly.score(X_test_poly, y_test)

        # Mostrar gráfico
        fig_poly = px.scatter(x=X_test[x_var], y=y_test.values.flatten(), 
                              title=f'Regresión No Lineal: Grado {degree}', 
                              labels={'x': x_var, 'y': y_var}, 
                              color_discrete_sequence=['blue'])
        
        # Línea de predicción en rojo
        fig_poly.add_trace(go.Scatter(x=X_test[x_var], y=y_pred_poly, mode='lines', name='Predicción', line=dict(color='red')))

        st.plotly_chart(fig_poly)
        st.write(f"R²: {r_squared_poly}")
        st.write(f"MSE: {mse_poly}")

        # REGRESIÓN LOGÍSTICA
    elif regression_type == "Regresión Logística":
        # Variables binarias seleccionables
        binary_vars = ['host_is_superhost', 'instant_bookable', 'has_availability', 'host_has_profile_pic', 'host_identity_verified']
        y_var_logistic = st.sidebar.selectbox("Selecciona la variable Y para la regresión logística", options=binary_vars)

        # Eliminar filas con valores nulos en X y y
        df_logistic = df1[[x_var, y_var_logistic]].dropna()
        X_log = df_logistic[[x_var]]  # Matriz de características
        y_log = df_logistic[y_var_logistic]  # Variable objetivo

        # Dividir los datos para entrenamiento y prueba
        X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X_log, y_log, test_size=0.3, random_state=42)

        # Modelo de regresión logística
        model_logistic = LogisticRegression()
        model_logistic.fit(X_train_log, y_train_log)

        # Predicciones y métricas
        y_pred_log = model_logistic.predict(X_test_log)
        accuracy = accuracy_score(y_test_log, y_pred_log)
        cm = confusion_matrix(y_test_log, y_pred_log)

        # Matriz de confusión
        st.write(f"Precisión del modelo: {accuracy}")
        st.write("Matriz de Confusión:")
        st.write(cm)

        # Gráfico de matriz de confusión con Seaborn
        fig_cm, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
        st.pyplot(fig_cm)

Overwriting app.py
